In [ ]:
cd <your path>

# Begining

## Load modules

In [ ]:
from operator import itemgetter
import time
import sys
import getpass
from pathlib import Path
import pandas as pd
import numpy as np

import itertools

## Define the Parameters

In [ ]:
# PATH = '/scratch/work/courses/DSGA1004-2021/MSD'
PATH = './data'

STATS_PATH = Path('./saved_tuned_bias_model_stats')

WHOLE_DATA = False
FRACTION = 0.01
TOP=500
PREC_AT = 500

# Load the data

In [ ]:
if WHOLE_DATA:
    train_pd = pd.read_parquet(PATH+'/cf_train_trans.parquet')
    unique_user_index_val_pd = pd.read_parquet(PATH+'/unique_user_index_val.parquet')
else:
    train_pd = pd.read_parquet(PATH+'/cf_train_trans_{}.parquet'.format(str(FRACTION).replace('.','_')))
    unique_user_index_val_pd = pd.read_parquet(PATH+'/unique_user_index_val.parquet')
    unique_user_index_val_pd = unique_user_index_val_pd.sample(replace=False, frac=max(FRACTION,0.05)) 

val_pd = pd.read_parquet(PATH+'/cf_validation_trans.parquet')
print('Successfully loaded the data')

Successfully loaded the data


In [ ]:
train_pd.rename(columns={'user_index':'user','track_index':'item','count':'rating'}, inplace=True)
train_pd.head(2)

,user_id,rating,track_id,__index_level_0__,user,item
0,c2934b29d64e467297c608351ed9695ce62128bc,2,TRAUAQB128F92D2DE9,2055,857987,11485
1,c2934b29d64e467297c608351ed9695ce62128bc,1,TRNZIAP128F93437BF,2056,857987,207539


In [ ]:
# unique_user_index_val_pd = unique_user_index_val.toPandas()
unique_user_index_val_pd = np.array(unique_user_index_val_pd.values.tolist())
unique_user_index_val_list = list(unique_user_index_val_pd.reshape(-1))

In [ ]:
# val_pd = val.toPandas()
val_pd.rename(columns={'user_index':'user','track_index':'item','count':'rating'}, inplace=True)
val_pd.head(2)

,user_id,rating,track_id,__index_level_0__,user,item
0,0007140a3796e901f3190f12e9de6d7548d4ac4a,1,TRUFCYO128F422B898,0,109,300356
1,0007140a3796e901f3190f12e9de6d7548d4ac4a,2,TROBZPR128F14808FF,1,109,209116


# ALS Model lenskit

In [ ]:
# ! pip install lenskit

In [ ]:
from lenskit.algorithms import bias
from lenskit.algorithms import Recommender, item_knn
from lenskit import batch, topn, util

In [ ]:
items=[True]
users=[False]
damping = [10**i for i in range(-1,10)]   

params = [[a,b,c] for a,b,c in itertools.product(items, users, damping)]
# params.append([False, False, 0.0])
print('length of params',len(params))

stats_file = open(STATS_PATH/'stats_lenskit_{}.txt'.format(str(FRACTION).replace('.','_')), 'a', buffering=1)
print(' '.join(sys.argv))
print(' '.join(sys.argv), file=stats_file)

for items, users, damping in params:
    since = time.time()
    baseline_bias = bias.Bias(items=items, users=users, damping=damping)
    baseline_bias = Recommender.adapt(baseline_bias)

    model_bias = baseline_bias.fit(train_pd)
    time_to_fit = time.time() - since

    pred_recs_tracks_val_ext = batch.recommend(model_bias, unique_user_index_val_list, TOP)
    time_to_recommend = time.time() - since

    rla = topn.RecListAnalysis()
    rla.add_metric(topn.precision)
    results = rla.compute(pred_recs_tracks_val_ext, val_pd)
    precisionAt_k = results.precision.mean()
    time_to_eval = time.time() - since
    
    print("items:{}, users:{}, damping:{}, Metric: {}, time TTL: {}, fit: {}, rec: {}, evl: {}".format(
        items,users,damping,
        precisionAt_k, 
        time_to_eval, time_to_fit, time_to_recommend-time_to_fit, time_to_eval-time_to_recommend 
    ), file=stats_file)

    print("items:{}, users:{}, damping:{}, Metric: {}, time TTL: {}, fit: {}, rec: {}, evl: {}".format(
        items,users,damping,
        precisionAt_k, 
        time_to_eval, time_to_fit, time_to_recommend-time_to_fit, time_to_eval-time_to_recommend 
    ))


length of params 3
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-12952f01-c776-45f6-9c41-6b3ebe2e4319.json
items:True, users:False, damping:10000000, Metric: 0.002708000000000002, time TTL: 38.73024272918701, fit: 0.28790998458862305, rec: 9.279690265655518, evl: 29.16264247894287
items:True, users:False, damping:100000000, Metric: 0.002708000000000002, time TTL: 38.14877724647522, fit: 0.2864866256713867, rec: 9.291252374649048, evl: 28.571038246154785
items:True, users:False, damping:1000000000, Metric: 0.002708000000000002, time TTL: 37.435462951660156, fit: 0.24057459831237793, rec: 8.631808757781982, evl: 28.563079595565796
